In [1]:
import os 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import matplotlib as mpl 
# import missingno as msno
import seaborn as sns

from src import config

In [2]:
df = pd.read_csv(config.RAW_FILE_PATH / 'olist_order_items_dataset.csv')

print(f'''
{df.info()}
{print(df.isna().sum())}
{df.shape}
{df.head()}
''')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB
order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

None
None
(112650, 7)
                           order_id  order_item_id  \
0  00010242fe8c5a6d1ba2dd792cb16214              1   
1  00018f77f2f0320c557190d7a144bdd3              1

In [3]:
df = df.groupby('order_id').agg({'product_id': lambda x: list(x)})

In [4]:
!pip install pyspark
!pip install findspark

     |████████████████████████████████| 281.4 MB 34 kB/s s eta 0:00:01  |▎                               | 2.7 MB 5.0 MB/s eta 0:00:56     |█                               | 8.1 MB 5.0 MB/s eta 0:00:55     |█▏                              | 9.9 MB 5.0 MB/s eta 0:00:54     |██                              | 17.1 MB 13.6 MB/s eta 0:00:20     |██▏                             | 19.3 MB 13.6 MB/s eta 0:00:20     |████▊                           | 41.9 MB 15.9 MB/s eta 0:00:16     |██████▏                         | 54.3 MB 15.9 MB/s eta 0:00:15     |███████████▋                    | 101.7 MB 14.1 MB/s eta 0:00:13     |███████████▊                    | 102.6 MB 14.1 MB/s eta 0:00:13     |████████████▎                   | 108.3 MB 14.1 MB/s eta 0:00:13     |█████████████                   | 114.8 MB 14.1 MB/s eta 0:00:12     |██████████████                  | 123.2 MB 17.3 MB/s eta 0:00:10     |█████████████████▊              | 156.1 MB 14.3 MB/s eta 0:00:09     |██████████████████▉           

In [ ]:
!apt-get update --fix-missing
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease                        
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2830 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1515 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2290 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [982 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates/restricted amd64 Packages [1015 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [3262 kB]
Fetched 12.1 MB in 9s (1353 kB/s)                                              
Reading package lists... Done
debconf: delaying package configuration, since a

In [ ]:
import pyspark.sql.types as T 
import pyspark.sql.functions as F 
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

import pyspark.sql.functions as F
from pyspark.ml.fpm import FPGrowth

import findspark
findspark.init()
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"


In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
# conf = SparkConf().setAppName("base").setMaster("local[*]")
# spark = SparkContext(conf=conf).getOrCreate()
# spark.stop()

In [ ]:
df = spark.read.options(header='true').csv('../data/raw/olist_order_items_dataset.csv')

df.show(2)

In [ ]:
df = df.dropDuplicates(['order_id', 'product_id']).sort('order_id')

df.show(2)

In [ ]:
df = df.groupBy('order_id').agg(F.collect_list('product_id')).sort('order_id')

df.show(2)

In [ ]:
fpGrowth = FPGrowth(itemsCol='collect_list(product_id)', minSupport=0.00001, minConfidence=0.0001)

model = fpGrowth.fit(df)

In [ ]:
model.freqItemsets.show()
items = model.freqItemsets.toPandas()

model.associationRules.show()
rules = model.associationRules.toPandas()

In [ ]:
items.to_csv('./mba_freq_items.csv')
rules.to_csv('./mba_pattern.csv')